In [1]:
from modules.load import *

# -- Parameters
path_scalogram = 'D:/USC/00_data/scalograms/2000t_16f'
inclusion_option = 'healthy_insomnia'
verbose = True

fname_scalogram = 'scalogram_%s_%s.npy' % (path_scalogram.split('/')[-1], inclusion_option)
fname_df_demo = "df_demo_%s_%s.csv" % (path_scalogram.split('/')[-1], inclusion_option)

# -- Get df_demo
# 1. Init df_demo =================================================
df_demo = init_df_demo(verbose=verbose)

# 2. Select patients with scalograms ==============================
df_demo = find_subjects_with_scalograms(path_scalogram=path_scalogram, df=df_demo, verbose=verbose)

# 3. Add 'bai' column to the df_demo ==============================
df_demo = add_bai_to_df_demo(df=df_demo, verbose=verbose)
df_demo = add_bai_to_df_demo_healthy(df=df_demo, verbose=verbose)

# 4. Select columns (features) ====================================
df_demo = select_columns(df_demo, verbose=verbose)

# 5. Remove abnormal values for (ISI and AHI) =====================
df_demo = remove_abnormal_ahi_isi(df_demo, verbose=verbose)

# 6. Add custom features ==========================================
#   - N2 latency/TST, REM latency/TST
#   - if there is any abnormal value, the patient with that value will be recorded as NaN in the column
df_demo = addFeatures(df_demo, verbose=verbose)

# 7. Assign paradoxical criteria ==================================
df_demo = assign_paradoxical(df_demo, verbose=verbose)
df_demo_pre_exclusion = df_demo.copy()

# 8. Subject Exclusion ============================================
df_demo = subject_inclusion(df_demo, inclusion_option=inclusion_option, verbose=verbose)


Laod df_init.csv ... Successful !!



ImportError: cannot import name 'autoencoders' from 'modules' (d:\USC\01_code\insomnia_clustering\modules\__init__.py)

In [2]:
scalograms.shape

(1877, 16, 2000, 1)

In [ ]:
def load_scalograms(path_scalogram, df, resample_len=None, verbose=True):
    df_demo = df.copy()

    scalograms = []
    for id in tqdm(df_demo.index.to_list(), desc='     load scalograms'):
        f = h5.File(os.path.join(path_scalogram, id+'.h5'), 'r')
        scal = np.array(f['scalogram'])   
        
        # choose C4
        # shape = (16, 7, 1, 2000)
        # 0 ~ 6: F3, F4, C3, C4, O1, O2, sleep stage
        scal = scal[:, 1, :, :]
        scalograms.append(scal)
    
    if not(timeLen_unity(scalograms)):
        scalograms = resampling(list_data=scalograms, len_custom=resample_len)
        scalograms = np.array(scalograms)
        scalograms = scalograms.astype(np.float32)    
    else:
        scalograms = np.array(scalograms)
        scalograms = scalograms.astype(np.float32)
        scalograms = scalograms[:,:,0,:][:,:,:,np.newaxis]
    
    myPrint("INIT 9_Load Selected Scalograms", verbose=verbose)
    myPrint("       Shape of scalograms: {}".format(scalograms.shape), verbose=verbose)

    return scalograms